In [1]:
# -*- coding: utf-8 -*-
!pip install catboost
!pip install -U scikit-learn
!pip install numpy
!pip install pandas

In [2]:
import os
import random
import pandas as pd
import numpy as np
import sklearn
import catboost

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from datetime import datetime
import time

start_time = time.perf_counter()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [4]:
#전부 NAN값인 피쳐 제거
a=[]
A = pd.DataFrame(train_df.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_df.shape[0]].index)
train_df.drop(columns=a,inplace=True)
test_df.drop(columns=a, inplace=True)

#분산이 1인 피쳐 제거거
a=[]
for c in train_df.columns:
  if  train_df[c].nunique()==1:
    a.append(c)
train_df.drop(columns=a, inplace=True)
test_df.drop(columns=a, inplace=True)

In [5]:
# 새로생긴 피쳐
new_features = train_df.columns[-351:]

In [6]:
#같은 라인에서 같은 열에 값을 가지는 행들을 그룹
groups_lines=train_df.groupby('LINE')
line=0
for name, group in groups_lines:
    line+=1
    globals()['group_line{}_df'.format(line)] = groups_lines.get_group(name)
non_null_counts_3 = group_line3_df.notnull().sum()
non_null_cols_3 = non_null_counts_3[non_null_counts_3 > 0].index.tolist()

grouped3 = group_line3_df.groupby(group_line3_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped3:
    cnt+=1
    globals()['line3_{}_df'.format(cnt)] = grouped3.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols3_{}'.format(i)] = globals()['line3_{}_df'.format(i)].columns[globals()['line3_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols3_{}'.format(i)][globals()['non_null_cols3_{}'.format(i)].index('PRODUCT_ID')]

grouped4 = group_line4_df.groupby(group_line4_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped4:
    cnt+=1
    globals()['line4_{}_df'.format(cnt)] = grouped4.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols4_{}'.format(i)] = globals()['line4_{}_df'.format(i)].columns[globals()['line4_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols4_{}'.format(i)][globals()['non_null_cols4_{}'.format(i)].index('PRODUCT_ID')]

non_null_counts_5 = group_line5_df.notnull().sum()
non_null_cols_5 = non_null_counts_5[non_null_counts_5 > 0].index.tolist()

grouped5 = group_line5_df.groupby(group_line5_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped5:
    cnt+=1
    globals()['line5_{}_df'.format(cnt)] = grouped5.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols5_{}'.format(i)] = globals()['line5_{}_df'.format(i)].columns[globals()['line5_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols5_{}'.format(i)][globals()['non_null_cols5_{}'.format(i)].index('PRODUCT_ID')]

grouped6 = group_line6_df.groupby(group_line6_df.isnull().apply(tuple, axis=1))
cnt=0
for name, group in grouped6:
    cnt+=1
    globals()['line6_{}_df'.format(cnt)] = grouped6.get_group(name)
for i in range(1,cnt+1):
  globals()['non_null_cols6_{}'.format(i)] = globals()['line6_{}_df'.format(i)].columns[globals()['line6_{}_df'.format(i)].notnull().all()].tolist()
  del globals()['non_null_cols6_{}'.format(i)][globals()['non_null_cols6_{}'.format(i)].index('PRODUCT_ID')]

In [7]:
#1, 2번 라인의 피쳐들
col_12 = train_df.columns.tolist()
col_12.remove('PRODUCT_ID')
print(len(col_12))
#################################
# 3,4 번 라인의 피쳐들
#col_34 = list(set(non_null_cols3_3+non_null_cols4_1))
col_34 =[]
for j in non_null_cols3_3:
    if j not in col_34:
        col_34.append(j)
for j in non_null_cols4_1:
    if j not in col_34:
        col_34.append(j)
col_34.append('X_2871')
print(len(col_34))
#####################################
# 5,6 번 라인의 피쳐들(그룹 1)
col_5 = set(non_null_cols5_1).intersection(set(non_null_cols5_2), set(non_null_cols5_3),set(non_null_cols5_4),set(non_null_cols5_5))
col_6 = set(non_null_cols6_1).intersection(set(non_null_cols6_2), set(non_null_cols6_3),set(non_null_cols6_4))
line_5 = [x for x in non_null_cols5_1 if x in col_5]
line_6 = [x for x in non_null_cols6_1 if x in col_6]

col_56_1 =[]
for j in line_5:
    if j not in col_56_1:
        col_56_1.append(j)
for j in line_6:
    if j not in col_56_1:
        col_56_1.append(j)
print(len(col_56_1))
#########################################
# 5,6 번 라인의 피쳐들(그룹 2)
col_5 = set(non_null_cols5_7).intersection(set(non_null_cols5_8), set(non_null_cols5_9),set(non_null_cols5_10),set(non_null_cols5_11))
col_6 = set(non_null_cols6_5).intersection(set(non_null_cols6_6),set(non_null_cols6_6),set(non_null_cols6_8),set(non_null_cols6_9))
line_5 = [x for x in non_null_cols5_7 if x in col_5]
line_6 = [x for x in non_null_cols6_5 if x in col_6]

col_56_2 =[]
for j in line_5:
    if j not in col_56_2:
        col_56_2.append(j)
for j in line_6:
    if j not in col_56_2:
        col_56_2.append(j)
print(len(col_56_2))

2794
1778
553
368


In [8]:
#1,2 번 라인 피쳐들, 학습에 사용할 df 처리
col_12_train = col_12
train_x = train_df[col_12_train]
col_12_test = col_12
del col_12_test[col_12_test.index('Y_Quality')]
del col_12_test[col_12_test.index('Y_Class')]
test_x = test_df[col_12_test]
train_x12 = train_x[train_x['LINE'].isin(['T010305','T010306'])]
test_x12 = test_x[test_x['LINE'].isin(['T010305','T010306'])]
#################################################################
#3,4 번 라인 피쳐들, 학습에 사용할 df 처리
col_34_train = col_34
train_x = train_df[col_34_train]
col_34_test = col_34
del col_34_test[col_34_test.index('Y_Quality')]
del col_34_test[col_34_test.index('Y_Class')]
test_x = test_df[col_34_test]

train_x34 = train_x[train_x['LINE'].isin(['T050304','T050307'])]
test_x34 = test_x[test_x['LINE'].isin(['T050304','T050307'])]
##################################################################
#5,6 번 라인(그룹1) 피쳐들, 학습에 사용할 df 처리
col_56_1_train = col_56_1
train_x = train_df[col_56_1_train]
col_56_1_test = col_56_1
del col_56_1_test[col_56_1_test.index('Y_Quality')]
del col_56_1_test[col_56_1_test.index('Y_Class')]
test_x = test_df[col_56_1_test]

train_x56_1 = train_x[train_x['LINE'].isin(['T100304','T100306'])]
test_x56_1 = test_x[test_x['LINE'].isin(['T100304','T100306'])]
######################################################################
#5,6 번 라인(그룹2) 피쳐들, 학습에 사용할 df 처리
col_56_2_train = col_56_2
train_x = train_df[col_56_2_train]
col_56_2_test = col_56_2
del col_56_2_test[col_56_2_test.index('Y_Quality')]
del col_56_2_test[col_56_2_test.index('Y_Class')]
test_x = test_df[col_56_2_test]

train_x56_2 = train_x[train_x['LINE'].isin(['T100304','T100306'])]
test_x56_2 = test_x[test_x['LINE'].isin(['T100304','T100306'])]

In [9]:
# LINE과 PRODUCT CODE 라벨링
train_x = train_df.drop(columns=['PRODUCT_ID'])
test_x = test_df.drop(columns=['PRODUCT_ID'])

qual_col = ['LINE', 'PRODUCT_CODE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 

train_y = train_x['Y_Quality']
train_yy = train_x['Y_Class']
train_x.drop(columns=['Y_Quality','Y_Class'],inplace=True)

In [10]:
# 5번과 6번라인의 두개의 그룹으로 데이터를 나눔
train_x56 = train_x[(train_x['LINE'] == 4) | (train_x['LINE'] == 5)]
col56_1_index=[]
col56_2_index=[]
for i in range(0,train_x56[new_features].shape[0]):
    if train_x56[new_features].iloc[i].sum() == 0:
        col56_1_index.append(i)
    else:
        col56_2_index.append(i)
train_x56_1 = train_x56_1.iloc[col56_1_index]
train_x56_2 = train_x56_2.iloc[col56_2_index]

In [11]:
#학습에 사용할 DF 처리
a=[]
A = pd.DataFrame(train_x12.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_x12.shape[0]].index)
train_x12.drop(columns=a,inplace=True)
drop_col_12 = a

yy_train12 = train_x12['Y_Class']
y_train12 = train_x12['Y_Quality']
train_x12 = train_x12.drop(columns=['Y_Class', 'Y_Quality','LINE','PRODUCT_CODE'])
test_x12= test_x12.drop(columns=['LINE','PRODUCT_CODE'])
########################################################

yy_train34 = train_x34['Y_Class']
y_train34 = train_x34['Y_Quality']
train_x34 = train_x34.drop(columns=['Y_Class', 'Y_Quality','LINE','PRODUCT_CODE'])
test_x34 = test_x34.drop(columns=['LINE','PRODUCT_CODE'])
############################################################

yy_train56_1 = train_x56_1['Y_Class']
y_train56_1 = train_x56_1['Y_Quality']
train_x56_1 = train_x56_1.drop(columns=['Y_Class', 'Y_Quality','LINE','PRODUCT_CODE'])
test_x56_1 = test_x56_1.drop(columns=['LINE','PRODUCT_CODE'])
############################################################

yy_train56_2 = train_x56_2['Y_Class']
y_train56_2 = train_x56_2['Y_Quality']
train_x56_2 = train_x56_2.drop(columns=['Y_Class', 'Y_Quality','LINE','PRODUCT_CODE'])
test_x56_2 = test_x56_2.drop(columns=['LINE','PRODUCT_CODE'])

In [12]:
# 총 4개의 모델 학습 (1,2 번라인),(3,4번라인),(5,6번 라인_그룹1),(5,6번 라인_그룹2)
reg = catboost.CatBoostRegressor(random_seed=37, learning_rate=0.05, verbose=0)
reg.fit(train_x12, y_train12, early_stopping_rounds=100)

clf34 = catboost.CatBoostRegressor(random_seed=37, learning_rate=0.05, verbose=0)
clf34.fit(train_x34, y_train34, early_stopping_rounds=100)

clf56_1 = catboost.CatBoostRegressor(random_seed=37, learning_rate=0.05, verbose=0)
clf56_1.fit(train_x56_1, y_train56_1, early_stopping_rounds=100)

clf56_2 = catboost.CatBoostRegressor(random_seed=37, learning_rate=0.05, verbose=0)
clf56_2.fit(train_x56_2, y_train56_2, early_stopping_rounds=100)

In [ ]:
import math
preds = []
# 회귀로 구한 값을 다시 Y_Class로 분류하기 위해서 통계를 냄
a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])

#test데이터를 한 행씩 읽어온다. 
for i in range(test_x.shape[0]):
  s = test_x.iloc[i,:]
  df_s = s.to_frame().transpose()
  df_s['LINE'] = int(df_s['LINE'])
# 읽어온 한 줄의 test 데이터가 어떤 LINE에 속하는지에 따라 다른 모델을 적용한다. 
  if int(s['LINE'])==0 or int(s['LINE'])==1 : #1,2번 line
      df_s = df_s[col_12_test]
      df_s = df_s.drop(columns=['LINE','PRODUCT_CODE'])
      df_s = df_s.drop(columns=drop_col_12)
      temp = reg.predict(df_s)
      if temp<=a[('Y_Quality','min')][1]:
          preds.append(0)
      elif temp<=a[('Y_Quality','max')][1]:
          preds.append(1)
      else:                                                                                                                                                                                                                                                                                                                                                                        
          preds.append(2)
  elif int(s['LINE'])==2 or int(s['LINE'])==3 :
      df_s = df_s[col_34_test]
      df_s = df_s.drop(columns=['LINE','PRODUCT_CODE'])
      temp = clf34.predict(df_s)
      if temp<=a[('Y_Quality','min')][1]:
          preds.append(0)
      elif temp<=a[('Y_Quality','max')][1]:
          preds.append(1)
      else:
          preds.append(2)
  elif int(s['LINE'])==4 or int(s['LINE'])==5 :
      if df_s[new_features].sum().sum() == 0:
          df_s = df_s[col_56_1_test]
          df_s = df_s.drop(columns=['LINE','PRODUCT_CODE'])
          temp = clf56_1.predict(df_s)
          if temp<=a[('Y_Quality','min')][1]:
              preds.append(0)
          elif temp<=a[('Y_Quality','max')][1]:
              preds.append(1)
          else:
              preds.append(2)
      else:
          df_s = df_s[col_56_2_test]
          df_s = df_s.drop(columns=['LINE','PRODUCT_CODE'])
          temp = clf56_2.predict(df_s)
          if temp<=a[('Y_Quality','min')][1]:
              preds.append(0)
          elif temp<=a[('Y_Quality','max')][1]:
              preds.append(1)
          else:
              preds.append(2)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv('./sample_submission.csv', index=False)

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"코드 실행 시간: {elapsed_time:.4f}초")